In [1]:
from sentence_transformers import SentenceTransformer, util
import jieba, nltk, re

d:\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [3]:
sentences = ['我想要去加入一个经常旅游的团队。',
            '我们团队平时会偶尔去旅游的',
            '我是一个不喜欢旅游的人']

In [4]:
embeddings = model.encode(sentences)

In [5]:
embeddings.shape

(3, 384)

In [6]:
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding.shape)

Sentence: 我想要去加入一个经常旅游的团队。
Embedding: (384,)
Sentence: 我们团队平时会偶尔去旅游的
Embedding: (384,)
Sentence: 我是一个不喜欢旅游的人
Embedding: (384,)


In [7]:
sim = util.cos_sim(embeddings[0], embeddings[1])
print("{0:.4f}".format(sim.tolist()[0][0]))
sim = util.cos_sim(embeddings[2], embeddings[1])
print("{0:.4f}".format(sim.tolist()[0][0]))

0.6433
0.3285


In [8]:
# hits = util.semantic_search(queries_embeddings, corpus_embeddings, top_k=2)

In [9]:
job_text = '【岗位职责】：1.一起头脑风暴，基于一个现有应用场景，讨论更多产品细节或技术功能，或基于我们已有的技术内容，开动脑筋，协助挖掘更多的发明点。（每天参加两小时头脑风暴或者培训会议）2.根据我们提供的技术框架，查更多资料，补充一些技术细节。3.审阅我们已经写好的专利内容，润色语言，并撰写发明创新idea。发明领域包括但不限于跟物联网、信息、软件、电学、机械、生活等相关的应用，包括人工智能、出行、文旅、教育、医疗、酒店、办公、社区、租赁、运动、餐饮等等。你可以挑选一个工作方向进行完成。如果你有学习能力，我们会对你进行专业培训，让你熟悉发明甚至拥有自己的发明专利。【任职要求】：1.喜欢创新，头脑活跃，喜欢交流，乐于表达自己。2.偏理工科专业，软件、物联网、智能、计算机方向优先，优秀者可放宽至本科生；文科生但是学习过跟发明或者专利代理相关的知识的也可以尝试。3.有物联网或算法或信息分析能力的同学佳，不需要会编程，会写相关领域技术或者思维活跃有创造力就可以。【福利待遇】：1.创新挖掘，根据是否挖掘到成果，每次两小时，60-260元，根据挖掘到的想法数量质量决定。2.每一篇你参与创意挖掘或者撰写的发明，你都有机会作为共同发明人进行发明专利的发表。3.实习三个月，可以提供实习证明。4.公司提供专业培训、创新辅助系统、创意框架和撰写模板。5.参与头脑风暴或者创新培训的同学可以线上或线下参与（推荐线下），每周至少参与两次，每次至少两小时。'
hunter_text = '信息与计算科学专业，c,c++有一定的基础知识，能够使用python进行编程，在校学习过数据库原理，'

In [10]:
def get_seg_words(text : str):
    seg_list = jieba.lcut(text)
    interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', 
                    '，', '。', '：', '；', '？', '（', '）', '【', '】', '！', '￥', ' ', '、', '-']
    seg_list = [word for word in seg_list if word not in interpunctuations]
    stops = set(nltk.corpus.stopwords.words("chinese"))
    seg_list = [word for word in seg_list if word not in stops]
    return seg_list

In [11]:
job_sentences = get_seg_words(job_text)
hunter_sentences = get_seg_words(hunter_text)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\vmice\AppData\Local\Temp\jieba.cache
Loading model cost 0.650 seconds.
Prefix dict has been built successfully.


In [12]:
# job_text = job_text.replace('。', '. ')
# # res = nltk.tokenize.word_tokenize(job_text)
# print(job_text)

# sentence = nltk.sent_tokenize(job_text)
# print(sentence)
# # job_text = ''.join(re.findall(r'[\u4e00-\u9fa5]', job_text))

# seg_list = jieba.lcut(job_text)
# print(seg_list)

# interpunctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', 
#                     '，', '。', '：', '；', '？', '（', '）', '【', '】', '！', '￥', ' ']
# seg_list = [word for word in seg_list if word not in interpunctuations]
# print(seg_list)

# stops = set(nltk.corpus.stopwords.words("chinese"))
# seg_list = [word for word in seg_list if word not in stops]
# print(seg_list)
# # text = nltk.Text(seg_list)

# type_tag = nltk.pos_tag(seg_list)
# print(type_tag)

# cut_word = []
# for word in seg_list:
#     cut_word.append(nltk.stem.PorterStemmer().stem(word))
# print(cut_word)

In [13]:
# hunter_text = hunter_text.replace('，', '').replace(',', '')

In [14]:
# seg_list = jieba.lcut(hunter_text, cut_all=False)
# print(seg_list)

In [15]:
sentences = [job_sentences, hunter_sentences]
embeddings = model.encode(sentences)
sim = util.cos_sim(embeddings[0], embeddings[1])
print("{0:.4f}".format(sim.tolist()[0][0]))

0.6277


In [16]:
sentences = [job_text, hunter_text]
embeddings = model.encode(sentences)
sim = util.cos_sim(embeddings[0], embeddings[1])
print("{0:.4f}".format(sim.tolist()[0][0]))

0.4581
